In [43]:
import numpy as np
import keras
from keras.models import Sequential , Model
from keras.layers import Input , RepeatVector , TimeDistributed  , GRU , Bidirectional , LSTM , Dense
from keras import regularizers
import os
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from keras.models import model_from_json
from keras.datasets import mnist


gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [67]:
def bimodel(maxlen,word_vector):
    inputs = Input(shape=(maxlen , word_vector))
    encoded = Bidirectional(GRU(128 , return_sequences=True , activity_regularizer=regularizers.l2(0.01)))(inputs)
    encoded = Bidirectional(GRU(64, return_sequences=False , activity_regularizer=regularizers.l2(0.01)))(encoded)
    decoded = RepeatVector(maxlen)(encoded)
    decoded = Bidirectional(GRU(64 ,return_sequences=True , activity_regularizer=regularizers.l2(0.01)))(decoded)
    decoded = Bidirectional(GRU(word_vector ,return_sequences=True , activity_regularizer=regularizers.l2(0.01)))(decoded)
    output = TimeDistributed(Dense(word_vector))(decoded)
    sequence_autoencoder = Model(inputs, output)
    
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [68]:
def origin_model(maxlen , word_vector):
    inputs = Input(shape=(maxlen , word_vector))
    encoded = GRU(128 , return_sequences=True)(inputs)
    encoded = GRU(64, return_sequences=False)(encoded)
    decoded = RepeatVector(maxlen)(encoded)
    decoded = GRU(64 ,return_sequences=True)(decoded)
    decoded = GRU(word_vector ,return_sequences=True)(decoded)
    output = TimeDistributed(Dense(word_vector))(decoded)
    sequence_autoencoder = Model(inputs, output)
    
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [73]:
def double_model(maxlen , word_vector):
    inputs = Input(shape=(maxlen , word_vector))
    encoded = GRU(256 , return_sequences=True)(inputs)
    encoded = GRU(128, return_sequences=False)(encoded)
    decoded = RepeatVector(maxlen)(encoded)
    decoded = GRU(128,return_sequences=True)(decoded)
    decoded = GRU(word_vector ,return_sequences=True)(decoded)
    output = TimeDistributed(Dense(word_vector))(decoded)
    sequence_autoencoder = Model(inputs, output)
    
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [17]:
def example_model(timesteps , input_dim ,latent_dim):
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dim)(inputs)

    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)

    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [33]:
(x_train, _), (x_test, _) = mnist.load_data()

In [34]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28))  # 'channels_firtst'이미지 데이터 형식을 사용하는 경우 이를 적용
x_test = np.reshape(x_test, (len(x_test), 28, 28))  # 'ch

In [36]:
x_train.shape

(60000, 28, 28)

In [37]:
test_model = example_model(28,28,64)

In [38]:
test_model.summary()
test_model.compile(optimizer='adadelta', loss='binary_crossentropy')

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                23808     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 28, 64)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 28, 28)            10416     
Total params: 34,224
Trainable params: 34,224
Non-trainable params: 0
_________________________________________________________________


In [40]:
test_model.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
)

60000/60000 [==============================] - ETA: 7s - loss: 0.223 - ETA: 7s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 4s - loss: 0.223 - ETA: 4s - loss: 0

In [53]:
bimodel = bimodel(28,28)
bimodel.compile(optimizer='adam', loss='mae')

In [54]:
bimodel.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 28, 256)           120576    
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 128)               123264    
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 28, 128)           0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 28, 128)           74112     
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 28, 56)            26376     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 28, 28)            159

In [55]:
bimodel.fit(x_train, x_train,
                epochs=10,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test),
)

60000/60000 [==============================] - ETA: 29s - loss: 0.42 - ETA: 29s - loss: 0.42 - ETA: 28s - loss: 0.42 - ETA: 28s - loss: 0.42 - ETA: 28s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 24s - loss: 0.42 - ETA: 24s - loss: 0.42 - ETA: 24s - loss: 0.41 - ETA: 24s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 19s - loss: 0.41 - ETA: 19s - loss: 0.41 - ETA: 19s - loss: 

In [70]:
origin_model1 = origin_model(28,28)

origin_model1.compile(optimizer='adam', loss='mae')

In [71]:
origin_model1.summary()

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
gru_33 (GRU)                 (None, 28, 128)           60288     
_________________________________________________________________
gru_34 (GRU)                 (None, 64)                37056     
_________________________________________________________________
repeat_vector_14 (RepeatVect (None, 28, 64)            0         
_________________________________________________________________
gru_35 (GRU)                 (None, 28, 64)            24768     
_________________________________________________________________
gru_36 (GRU)                 (None, 28, 28)            7812      
_________________________________________________________________
time_distributed_8 (TimeDist (None, 28, 28)            812

In [72]:
origin_model1.fit(x_train, x_train,
                epochs=10,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test),
)

60000/60000 [==============================] - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 9s - loss: 0

In [74]:
double_model = double_model(28,28)
double_model.compile(optimizer='adam', loss='mae')

In [75]:
double_model.summary()

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
gru_37 (GRU)                 (None, 28, 256)           218880    
_________________________________________________________________
gru_38 (GRU)                 (None, 128)               147840    
_________________________________________________________________
repeat_vector_15 (RepeatVect (None, 28, 128)           0         
_________________________________________________________________
gru_39 (GRU)                 (None, 28, 128)           98688     
_________________________________________________________________
gru_40 (GRU)                 (None, 28, 28)            13188     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 28, 28)            812

In [76]:
double_model.fit(x_train, x_train,
                epochs=10,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test),
)

60000/60000 [==============================] - ETA: 15s - loss: 0.08 - ETA: 15s - loss: 0.08 - ETA: 15s - loss: 0.07 - ETA: 15s - loss: 0.07 - ETA: 15s - loss: 0.07 - ETA: 15s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 